In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid_spec
import plotly.graph_objects as go
import lightgbm as lgbm  
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold, GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.impute import KNNImputer
from sklearn import linear_model
from sklearn.linear_model import HuberRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV
import joblib

import warnings
warnings.filterwarnings('ignore')

In [3]:
df_train = pd.read_csv('/content/drive/MyDrive/tabular-playground-series-aug-2022/train.csv')# read data
df_test = pd.read_csv('/content/drive/MyDrive/tabular-playground-series-aug-2022/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/tabular-playground-series-aug-2022/sample_submission.csv')
df_train.describe()#Returns the statistical variables of this core data structure


,id,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
count,26570.000000,26320.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26189.000000,26032.000000,25894.000000,...,25343.000000,25270.000000,25102.000000,24969.000000,24796.000000,24696.000000,24561.000000,24460.000000,24286.000000,26570.000000
mean,13284.500000,127.826233,6.754046,7.240459,7.415883,8.232518,6.256568,17.791528,11.731988,17.127804,...,11.430725,16.117711,19.172085,11.702464,15.652904,16.048444,14.995554,16.460727,701.269059,0.212608
std,7670.242662,39.030020,1.471852,1.456493,4.116690,4.199401,3.309109,1.001200,0.996085,0.996414,...,0.999137,1.405978,1.520785,1.488838,1.155247,1.491923,1.549226,1.708935,123.304161,0.409160
min,0.000000,33.160000,5.000000,5.000000,0.000000,0.000000,0.000000,13.968000,8.008000,12.073000,...,7.537000,9.323000,12.461000,5.167000,10.890000,9.140000,9.104000,9.701000,196.787000,0.000000
25%,6642.250000,99.987500,6.000000,6.000000,4.000000,5.000000,4.000000,17.117000,11.051000,16.443000,...,10.757000,15.209000,18.170000,10.703000,14.890000,15.057000,13.957000,15.268000,618.961500,0.000000
50%,13284.500000,122.390000,6.000000,8.000000,7.000000,8.000000,6.000000,17.787000,11.733000,17.132000,...,11.430000,16.127000,19.211500,11.717000,15.628500,16.040000,14.969000,16.436000,701.024500,0.000000
75%,19926.750000,149.152500,8.000000,8.000000,10.000000,11.000000,8.000000,18.469000,12.410000,17.805000,...,12.102000,17.025000,20.207000,12.709000,16.374000,17.082000,16.018000,17.628000,784.090250,0.000000
max,26569.000000,385.860000,9.000000,9.000000,29.000000,29.000000,24.000000,21.499000,16.484000,21.425000,...,15.412000,22.479000,25.640000,17.663000,22.713000,22.303000,21.626000,24.094000,1312.794000,1.000000


In [140]:
target_class = pd.DataFrame({'count': df_train.failure.value_counts(), 'percentage': df_train['failure'].value_counts() / df_train.shape[0] * 100})
target_class#Observe the ratio of 1 and 0

,count,percentage
0,20921,78.73918
1,5649,21.26082


In [4]:
Z = df_train['failure']
df_train.drop('failure',axis=1, inplace = True)#remove"failure"
Data = pd.concat([df_train, df_test])# use concat to merge


In [142]:
df_train.isna().sum()#Count the number of occurrences of NaN in a column of Pandas Dataframe


id                   0
product_code         0
loading            250
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      381
measurement_4      538
measurement_5      676
measurement_6      796
measurement_7      937
measurement_8     1048
measurement_9     1227
measurement_10    1300
measurement_11    1468
measurement_12    1601
measurement_13    1774
measurement_14    1874
measurement_15    2009
measurement_16    2110
measurement_17    2284
dtype: int64

In [7]:
df_test.isna().sum()#Count the number of occurrences of NaN in a column of Pandas Dataframe

id                   0
product_code         0
loading            223
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      329
measurement_4      409
measurement_5      508
measurement_6      624
measurement_7      720
measurement_8      846
measurement_9      904
measurement_10    1067
measurement_11    1136
measurement_12    1240
measurement_13    1303
measurement_14    1440
measurement_15    1542
measurement_16    1678
measurement_17    1740
dtype: int64

In [5]:
! pip install feature_engine
from feature_engine.encoding import WoEEncoder
#The WoEEncoder() replaces categories by the weight of evidence

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 KB 7.5 MB/s eta 0:00:00


In [6]:
Data['missing_3'] = Data['measurement_3'].isnull().astype(np.int16)#Use astype() data type to convert to np.int16
Data['missing_5'] = Data['measurement_5'].isnull().astype(np.int16)
'''
the conditional failure rates of missing measurement_3 and missing measurement_5 deviate significantly from the average failure rate
e can use the features missing_3 and missing_5 in our models
'''

'\nthe conditional failure rates of missing measurement_3 and missing measurement_5 deviate significantly from the average failure rate\ne can use the features missing_3 and missing_5 in our models\n'

In [7]:

def Corr(d,x):
  #Correlation between two variables in data
  return  np.absolute(d.corr()[f'{x}']).sort_values(ascending=False)
def kk(t,c,m):
  train = t[c+[m]].dropna(how='any')
  test = t[(t[c].isnull().sum(axis=1)==0)&(t[m].isnull())]
  return train,test

In [8]:

Feature = [k for k in df_test.columns if k.startswith('measurement') or k=='loading']
#Treat words with "measurement" and "loading" as Feature
FullDict ={}
other = [other for other in df_test.columns if other.startswith('attribute')or other=='id']+ ['loading','product_code','missing_3','missing_5']
s = []
n =[]
for x in range(3,18):  
    corr =Corr(Data.drop(other, axis=1),f'measurement_{x}')
    s.append(np.round(np.sum(corr[1:5]),5)) 
    # add the 5 first lines of the correlation 
    n.append(f'measurement_{x}')
show = pd.DataFrame()
show['columns'] = n
show['correlation'] = s #most correlated
show = show.sort_values(by = 'correlation',ascending=False).reset_index(drop = True)



In [9]:
for i in range(8):
    next = show.iloc[i,0] 
    # select the next best correlated column 
    Fill ={}
    for x in Data.product_code.unique() : 
      NextDic = {}
      corr = Corr(Data[Data.product_code == x],next)
      NextDic[next] = corr[1:5].index.tolist()
      Fill[x] = NextDic[next]
    FullDict[next] =Fill


In [10]:
Feature = [k for k in Data.columns if k.startswith('measurement') or k=='loading']
nulls = [col for col in df_train.columns if df_train[col].isnull().sum()!=0]
for i in Data.product_code.unique():
    TotalNa_filled = 0
    for measure_col in list(FullDict.keys()):
        model = HuberRegressor(epsilon=1.35, max_iter = 400)
        t = Data[Data.product_code==i]
        column = FullDict[measure_col][i]
        ttrain,ttest= kk(t,column,measure_col)
        model.fit(ttrain[column], ttrain[measure_col])
        Data.loc[(Data.product_code==i)&(Data[column].isnull().sum(axis=1)==0)&(Data[measure_col].isnull()),measure_col] = model.predict(ttest[column])
        TotalNa_filled += len(ttest)
    NA = Data.loc[Data["product_code"] == i,nulls].isnull().sum().sum()
    model1 = KNNImputer(n_neighbors=3)
    Data.loc[Data.product_code==i,Feature] = model1.fit_transform(Data.loc[Data.product_code==i, Feature])
    joblib.dump(model1, 'KNN_model')

In [11]:
def scale(train_data, val_data, test_data, feats):
    scaler = StandardScaler()
    scaled_train = scaler.fit_transform(train_data[feats])
    scaled_val = scaler.transform(val_data[feats])
    scaled_test = scaler.transform(test_data[feats])
    new_train = train_data.copy()
    new_val = val_data.copy()
    new_test = test_data.copy()
    new_train[feats] = scaled_train
    new_val[feats] = scaled_val
    new_test[feats] = scaled_test
    return new_train, new_val, new_test

In [12]:
LE= LabelEncoder()
Data['attribute_1'] = LE.fit_transform(Data['attribute_1'])

In [13]:
df_train = Data.iloc[:df_train.shape[0],:]
df_test = Data.iloc[df_train.shape[0]:,:]
print(df_train.shape, df_test.shape)

G = df_train.product_code
X = df_train


(26570, 27) (20775, 27)


In [14]:
woe = WoEEncoder(variables=['attribute_0'])
woe.fit(X, Z)
w = woe.transform(X)
test = woe.transform(df_test)

In [15]:
class Train3_Valid2():
    def __init__(self):
        self.name = "Splitter with 2 product code in validation set"
        
    def split(self, X: pd.DataFrame, _y=None, _groups=None):
        indices = list(X.groupby("product_code").indices.values())
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                yield [np.concatenate([ix for k, ix in enumerate(indices) if k not in [i, j]]),
                    np.concatenate([ix for k, ix in enumerate(indices) if k in [i, j]])]

In [16]:

select = ['loading','attribute_0','measurement_17','measurement_0',
          'measurement_1','measurement_2','missing_3','missing_5',]

In [18]:
import pickle
lr_test = np.zeros(len(test))
cv = Train3_Valid2()
kf = GroupKFold(n_splits=5)
SPLIT = 5
lr_oof = np.zeros(len(df_train))
lr_test = np.zeros(len(df_test))
lr_auc = 0
importance_list = []
for fold_idx, (train_idx, val_idx) in enumerate(kf.split( w, Z, G)):# = train.product_code 
    x_train, x_val = w.iloc[train_idx], w.iloc[val_idx]
    y_train, y_val = Z.iloc[train_idx], Z.iloc[val_idx]
    x_train, x_val, x_test = scale(x_train, x_val, test, select)

    model = linear_model.LogisticRegression(max_iter=200, C=0.0001, penalty='l2', solver='newton-cg')
    model.fit(x_train[select], y_train)

with open('./x.pickle', 'wb') as f:
    pickle.dump(model, f)
   
